Dependencies

In [ ]:
# Install dependencies
!pip install python-dotenv
!pip install openai
!pip install openpyxl

# No personality traits prompt

In [ ]:
import os

import dotenv
import json
import re
import openai
import time
import sqlite3
import json

from dotenv import load_dotenv
load_dotenv()

from threading import Thread

openai.api_key = () # Set api key

MAX_FAILURES = 4


def create_prompt(scenario, personality, author):
    left_a, left_b = scenario[0]
    right_a, right_b = scenario[1]
    return f"""You are the fictional {personality}. 
- Your task is to make a decision in a game that determines a payment to you and Stranger A.
- Stranger A has no choice in this game. You and Stranger A do not meet and will remain anonymous.
- Make your decision using what you know of {personality} from the original {author} work and all subsequent commentary.
- In this game you have two options:

Option Left: Stranger A receives {left_a}, and you receive {left_b}, 
Option Right: Stranger A receives {right_a}, and you receive {right_b}, 

What do you choose, with one word [Left, Right]?

Output three columns: 1) the choice you made [Left or Right], 2) the amount you receive, 3) and the amount Stranger A receives. Do not output anything else. An example of the format of the output is: Left, 750, 500
"""
# Should change example amount based on scenarios


def get_decision(scenario, personality, author, scenario_name, model):
    prompt = create_prompt(scenario, personality, author)
    failure_count = 0
    message_history = []
    message_history.append({"role":"user", "content": prompt})
    # Get response
    completion = openai.ChatCompletion.create(
        model='gpt-4',
        messages=message_history,
    )
    reply_content1 = completion.choices[0].message.content

    return dict({"raw_reply": reply_content1,
                 "choice": "Left" if "left" in reply_content1.lower() else "Right", 
                 "scenario": scenario,
                 "personality":personality, # Character
                 "model":model,
                 "scenario_name":scenario_name,
                 "prompt":prompt
                })

# Personality traits prompt

In [ ]:
import os

import dotenv
import json
import re
import openai
import time
import sqlite3
import json

from dotenv import load_dotenv
load_dotenv()

from threading import Thread

openai.api_key = ('') # Set api key

MAX_FAILURES = 4



def create_prompt(scenario, personality, author):
    left_a, left_b = scenario[0]
    right_a, right_b = scenario[1]
    return (f"""You are the fictional {personality}. 
- Your task is to make a decision in a game that determines a payment to you and Person A.
- Person A has no choice in this game. You and Person A do not meet and will remain anonymous.
- Make your decision using what you know of {personality} from the original {author} work and all subsequent commentary.
- In this game you have two options.""",
            
f"""Option Left: Person A receives {left_a}, and you receive {left_b}. 
Option Right: Person A receives {right_a}, and you receive {right_b}.

What personality traits of {personality} from {author}  would affect {personality}'s decision? Output a list of the 5 most impactful personality traits with little to no explanation:""",
            
f"""Based on these personality traits, what would {personality} choose, with one word [Left, Right]?
Output two columns: 1) the choice you made [Left or Right], 2) the amount you receive. Do not output anything else. An example of the format of the output is: Right, 700""")



def get_decision(scenario, personality, author, scenario_name, model):
    prompt = create_prompt(scenario, personality, author)
    failure_count = 0
    message_history = []
    message_history.append({"role":"system", "content": prompt[0]}) # System settings
    message_history.append({"role":"user", "content": prompt[1]}) # Prompt 1
    # Get response
    completion = openai.ChatCompletion.create(
        model='gpt-4',
        messages=message_history,
    )
    reply_content1 = completion.choices[0].message.content
    message_history.append({"role":"assistant", "content": reply_content1})
    
    message_history.append({"role":"user", "content": prompt[2]}) # Prompt 2
    # Get response
    completion = openai.ChatCompletion.create(
        model='gpt-4',
        messages=message_history,
    )
    reply_content2 = completion.choices[0].message.content
    message_history.append({"role":"assistant", "content": reply_content2})

    return dict({"raw_reply": message_history,
                 "choice": "Left" if "left" in reply_content2.lower() else "Right", 
                 "scenario": scenario,
                 "personality":personality, # Character
                 "model":model,
                 "scenario_name":scenario_name,
                 "prompt":prompt[1],
                 "personalities": reply_content1
                })


Games to run:

In [ ]:
# Berk and Barc
scenarios = dict({ 
    "Berk29": ((400, 400), (750, 400)),
    "Barc2": ((400, 400), (750, 375)),
    "Berk23": ((800, 200), (0, 0)),
    "Barc8": ((300, 600), (700, 500)),
    "Berk15": ((200, 700), (600, 600)),
    "Berk26":((0, 800), (400, 400))
})

Choose characters and run experiment

In [ ]:
from time import sleep, perf_counter
import concurrent.futures

class DictatorGame():

    def __init__(self, scenario, personality, author, scenario_name, model):
        self.scenario = scenario
        self.personality = personality
        self.scenario_name = scenario_name
        self.model = model
        self.author = author

def run_game(args):
    game = args[0]
    err = 0
    while err < 5:
        try:
            decision = get_decision(scenario = game.scenario, personality = game.personality, author = game.author, scenario_name = game.scenario_name, model = game.model)
            return decision
        except Exception as e:
            err+=1
            sleep(240)
            print(f"Error #{err}: {e}")

model = 'gpt-4'
# List of personalities
personalities = []

# List of authors corresponding to personalities
authors = []

observations = list()

with concurrent.futures.ThreadPoolExecutor() as executor:
    for i, personality in enumerate(personalities):
        for scenario_name, scenario in scenarios.items():
            dictator_game = DictatorGame(scenario = scenario, personality = personality, author=authors[i], scenario_name = scenario_name, model = model)
            f1 = executor.submit(run_game, args=[dictator_game])
            observations.append(f1)
                
    # Check for completed results
    i = 0
    for f in concurrent.futures.as_completed(observations):
        print(f'{i}')
        i+=1

# With personality traits

In [ ]:
import pandas as pd

# Take observations and format the data into a pandas dataframe
def write_data1(obv=observations):
    df = pd.DataFrame(columns=['Model', 'Scenario', 'Character', 'Decision', 'RawData', 'Personalities'])
    for i, observation in enumerate(obv):
        result = observation.result()
        new_row = pd.DataFrame({'Model': [result['model']], 'Scenario' : [result['scenario']], 'Character' : [result['personality']], 
                   'Decision' : [result['choice']], 'Personalities' : [result['personalities']]})
        df = pd.concat([df, new_row], ignore_index=True)
    return df
df = write_data1(observations)    

# Without peronsality traits

In [ ]:
import pandas as pd

# Take observations and format the data into a pandas dataframe
def write_data2(observations):
    df = pd.DataFrame(columns=['Model', 'Scenario', 'Character', 'Decision', 'RawData'])
    for observation in observations:
        result = observation.result()
        new_row = pd.DataFrame({'Model': [result['model']], 'Scenario' : [result['scenario']], 'Character' : [result['personality']], 
                        'Decision' : [result['choice']], 'RawData' : [result['raw_reply']]})
        df = pd.concat([df, new_row], ignore_index=True)
    return df
df = write_data2(observations)    

In [ ]:
# Write data to excel file
df.to_excel('name.xlsx', index=False)